# Preparing RDS files for export for ETE visualization
End goal is to annotate each lineage with import driver genes that are mutated in it. In order to do this, we need to map phyloWGS output back to the SNV information from the parser (create_inputs_v3.R)


In [41]:
getwd()

[1] "/Volumes/yjiao-1/phylowgs2/inputs"

In [6]:
setwd('/Volumes/yjiao-1/phylowgs2/inputs/')

In [65]:
varfiles <- list.files(pattern = 'varGroups.rds')
sapply(varfiles, function(vf){
    outname <- paste(strsplit(vf, '\\.')[[1]][1], '.tsv', sep='')
    vars <- readRDS(vf)
    onerowvars <- lapply(vars, function(row) row[1,]) # just keep the first row of each group, we don't really care about the sample info
    vars <- do.call(rbind, onerowvars)
    write.table(vars, file=outname, sep='\t', row.names=FALSE, col.names = TRUE)
    'done'
})


122T_varGroups.rds   131T_varGroups.rds   148T_varGroups.rds 
              "done"               "done"               "done" 
  155T_varGroups.rds   208T_varGroups.rds   253T_varGroups.rds 
              "done"               "done"               "done" 
   25T_varGroups.rds   272T_varGroups.rds    39T_varGroups.rds 
              "done"               "done"               "done" 
   42T_varGroups.rds    51T_varGroups.rds    62T_varGroups.rds 
              "done"               "done"               "done" 
   98T_varGroups.rds    99T_varGroups.rds PD004T_varGroups.rds 
              "done"               "done"               "done" 
  T33T_varGroups.rds 
              "done"

In [64]:
list.files(pattern='.tsv')

[1] "122T_varGroups.tsv"   "131T_varGroups.tsv"   "148T_varGroups.tsv"  
 [4] "155T_varGroups.tsv"   "208T_varGroups.tsv"   "253T_varGroups.tsv"  
 [7] "25T_varGroups.tsv"    "272T_varGroups.tsv"   "39T_varGroups.tsv"   
[10] "42T_varGroups.tsv"    "51T_varGroups.tsv"    "62T_varGroups.tsv"   
[13] "98T_varGroups.tsv"    "99T_varGroups.tsv"    "PD004T_varGroups.tsv"
[16] "T33T_varGroups.tsv"

### Parsing CNV files
This is a bit more complicated, since now we also need to put the merged cnvs back
Files to use:
- **segGroups.rds** contains all the segs before filtering for quality
- **orderedSegs.rds** contains segs before merging
- **mergeflag.rds** tells us which segments were merged

In [7]:
getPat <- function(x){
    sapply(x, function(y) strsplit(y, '_')[[1]][1])
}

ordsegs <- list.files(pattern = 'orderedSegs.rds')
mergeflags <- list.files(pattern = 'mergeFlag.rds')

names(ordsegs) <- getPat(ordsegs)
names(mergeflags) <- getPat(mergeflags)

#sapply(names(ordsegs), function(pat){
    pat <- '39T'

    ords <- readRDS(ordsegs[pat])
    ords <- lapply(ords, function(x) x[1,])
    ords <- do.call(rbind, ords)

    merges <- readRDS(mergeflags[pat])
    notmerged_end <- sum(!merges)
        
    ords$cnvID <- NaN
    ords[!merges, 'cnvID'] <- paste('c', 0:(notmerged_end-1), sep='')
    ords[merges, 'cnvID'] <- paste('c', notmerged_end, sep='')
#})
#strsplit(segs[[732]][1,]$intersectingGenes, ',')
#strsplit(segs[[733]][1,]$intersectingGenes, ',')
#sapply(varfiles, function(vf){
#    outname <- paste(strsplit(vf, '\\.')[[1]][1], '.tsv', sep='')
#    vars <- readRDS(vf)
#    onerowvars <- lapply(vars, function(row) row[1,]) # just keep the first row of each group, we don't really care about the sample info
#    vars <- do.call(rbind, onerowvars)
#    write.table(vars, file=outname, sep='\t', row.names=FALSE, col.names = TRUE)
#    'done'
#})


In [12]:
dispidx <- names(ords) != 'intersectingGenes'
ords[ords$Chromosome == '7' & ords$Start.bp <= 140453136 & ords$End.bp >= 140453136, dispidx]
test <- readRDS('39T_cnvout.rds')
test[test$cnv=='c696',]
rownames(test)
test$cnv

,sample,Chromosome,Start.bp,End.bp,n_probes,length,seg_sigma,W,total_copy_ratio,modal_total_cn,ellip.h,SC_HZ,amp.a1,amp.a2,rescaled.cn.a1,rescaled.cn.a2,oldstart,oldstop,ID,prevalence,cnvID
7_100609625_1_2,SM-61EWU,7,100609625,141765404,6730,141571903,0.00639,0.05102153,0.9577,3,⋯,0,0,0,1,2,193501,141765404,7_100609625_1_2,0,c696
7_100609625_2_2,SM-61EWU,7,100609625,141765404,2269,41851170,0.00426,0.01526,1.12647,4,⋯,0,0,0,2,2,100609625,142460795,7_100609625_2_2,1,c697


,cnv,a,d,ssms
798,c696,"45292900,45292900,45292900,45292900,45292900,45292900,22646450","45292900,45292900,45292900,45292900,45292900,45292900,45292900","s372,1,3"


[1] "1"    "2"    "3"    "4"    "5"    "6"    "7"    "8"    "9"    "10"  
 [11] "11"   "12"   "13"   "14"   "15"   "16"   "17"   "18"   "19"   "20"  
 [21] "21"   "22"   "23"   "24"   "25"   "26"   "27"   "28"   "29"   "30"  
 [31] "31"   "32"   "33"   "34"   "35"   "36"   "37"   "38"   "39"   "40"  
 [41] "41"   "42"   "43"   "44"   "45"   "46"   "47"   "48"   "49"   "50"  
 [51] "51"   "52"   "53"   "54"   "55"   "56"   "57"   "58"   "59"   "60"  
 [61] "61"   "62"   "63"   "64"   "65"   "66"   "67"   "68"   "69"   "70"  
 [71] "71"   "72"   "73"   "74"   "75"   "76"   "77"   "78"   "79"   "80"  
 [81] "81"   "82"   "83"   "84"   "85"   "86"   "87"   "88"   "89"   "90"  
 [91] "91"   "92"   "93"   "94"   "95"   "96"   "97"   "98"   "99"   "100" 
[101] "101"  "102"  "103"  "104"  "105"  "106"  "107"  "108"  "109"  "110" 
[111] "111"  "112"  "113"  "114"  "115"  "116"  "117"  "118"  "119"  "120" 
[121] "121"  "122"  "123"  "124"  "125"  "126"  "127"  "128"  "129"  "130" 
[131] "131"  "132"  "133"  "134"  "135"  "136"  "137"  "138"  "139"  "140" 
[141] "141"  "142"  "143"  "144"  "145"  "146"  "147"  "148"  "149"  "150" 
[151] "151"  "152"  "153"  "154"  "155"  "156"  "157"  "158"  "159"  "160" 
[161] "161"  "162"  "163"  "164"  "165"  "166"  "167"  "168"  "169"  "170" 
[171] "171"  "172"  "173"  "174"  "175"  "176"  "177"  "178"  "179"  "180" 
[181] "181"  "182"  "183"  "184"  "185"  "186"  "188"  "189"  "192"  "193" 
[191] "202"  "203"  "204"  "208"  "212"  "213"  "214"  "215"  "216"  "217" 
[201] "218"  "219"  "220"  "221"  "222"  "223"  "224"  "225"  "226"  "227" 
[211] "228"  "229"  "230"  "231"  "232"  "233"  "234"  "235"  "236"  "237" 
[221] "238"  "239"  "240"  "241"  "242"  "243"  "244"  "245"  "246"  "247" 
[231] "248"  "249"  "250"  "251"  "252"  "253"  "254"  "255"  "256"  "257" 
[241] "258"  "259"  "260"  "261"  "262"  "263"  "264"  "265"  "266"  "267" 
[251] "268"  "269"  "270"  "271"  "272"  "273"  "274"  "275"  "276"  "277" 
[261] "278"  "279"  "280"  "285"  "286"  "287"  "289"  "290"  "291"  "292" 
[271] "293"  "294"  "295"  "302"  "306"  "307"  "308"  "312"  "313"  "314" 
[281] "315"  "316"  "317"  "318"  "319"  "320"  "321"  "322"  "323"  "324" 
[291] "325"  "326"  "327"  "328"  "329"  "330"  "331"  "332"  "333"  "334" 
[301] "335"  "336"  "337"  "338"  "339"  "340"  "341"  "342"  "343"  "344" 
[311] "345"  "346"  "347"  "348"  "349"  "350"  "351"  "352"  "353"  "354" 
[321] "355"  "356"  "357"  "358"  "359"  "360"  "361"  "362"  "363"  "364" 
[331] "365"  "366"  "367"  "368"  "369"  "370"  "371"  "372"  "373"  "374" 
[341] "375"  "376"  "377"  "379"  "380"  "381"  "382"  "383"  "385"  "386" 
[351] "387"  "388"  "389"  "390"  "391"  "392"  "393"  "394"  "395"  "396" 
[361] "397"  "398"  "399"  "400"  "401"  "402"  "403"  "404"  "405"  "406" 
[371] "407"  "408"  "409"  "410"  "411"  "412"  "413"  "414"  "415"  "416" 
[381] "417"  "418"  "419"  "420"  "421"  "422"  "423"  "424"  "425"  "426" 
[391] "427"  "428"  "429"  "430"  "431"  "432"  "433"  "434"  "435"  "436" 
[401] "437"  "438"  "439"  "440"  "441"  "442"  "443"  "444"  "445"  "446" 
[411] "447"  "448"  "449"  "450"  "452"  "453"  "454"  "455"  "456"  "457" 
[421] "459"  "460"  "461"  "462"  "463"  "464"  "465"  "466"  "467"  "469" 
[431] "474"  "475"  "477"  "478"  "479"  "480"  "481"  "482"  "488"  "489" 
[441] "490"  "491"  "492"  "493"  "496"  "497"  "498"  "499"  "500"  "501" 
[451] "502"  "503"  "504"  "505"  "506"  "507"  "508"  "510"  "511"  "512" 
[461] "513"  "514"  "515"  "516"  "517"  "518"  "519"  "520"  "521"  "522" 
[471] "523"  "524"  "525"  "526"  "527"  "528"  "529"  "530"  "531"  "532" 
[481] "535"  "536"  "537"  "538"  "539"  "540"  "542"  "544"  "545"  "546" 
[491] "547"  "548"  "549"  "550"  "551"  "553"  "554"  "556"  "557"  "558" 
[501] "559"  "562"  "568"  "569"  "570"  "571"  "572"  "573"  "577"  "578" 
[511] "579"  "580"  "581"  "582"  "584"  "585"  "587"  "588"  "591"  "592" 
[521] "593"  "594"  "595"  "596"  "597"  "598"  "5

[1] "c0"   "c1"   "c2"   "c3"   "c4"   "c5"   "c6"   "c7"   "c8"   "c9"  
 [11] "c10"  "c11"  "c12"  "c13"  "c14"  "c15"  "c16"  "c17"  "c18"  "c19" 
 [21] "c20"  "c21"  "c22"  "c23"  "c24"  "c25"  "c26"  "c27"  "c28"  "c29" 
 [31] "c30"  "c31"  "c32"  "c33"  "c34"  "c35"  "c36"  "c37"  "c38"  "c39" 
 [41] "c40"  "c41"  "c42"  "c43"  "c44"  "c45"  "c46"  "c47"  "c48"  "c49" 
 [51] "c50"  "c51"  "c52"  "c53"  "c54"  "c55"  "c56"  "c57"  "c58"  "c59" 
 [61] "c60"  "c61"  "c62"  "c63"  "c64"  "c65"  "c66"  "c67"  "c68"  "c69" 
 [71] "c70"  "c71"  "c72"  "c73"  "c74"  "c75"  "c76"  "c77"  "c78"  "c79" 
 [81] "c80"  "c81"  "c82"  "c83"  "c84"  "c85"  "c86"  "c87"  "c88"  "c89" 
 [91] "c90"  "c91"  "c92"  "c93"  "c94"  "c95"  "c96"  "c97"  "c98"  "c99" 
[101] "c100" "c101" "c102" "c103" "c104" "c105" "c106" "c107" "c108" "c109"
[111] "c110" "c111" "c112" "c113" "c114" "c115" "c116" "c117" "c118" "c119"
[121] "c120" "c121" "c122" "c123" "c124" "c125" "c126" "c127" "c128" "c129"
[131] "c130" "c131" "c132" "c133" "c134" "c135" "c136" "c137" "c138" "c139"
[141] "c140" "c141" "c142" "c143" "c144" "c145" "c146" "c147" "c148" "c149"
[151] "c150" "c151" "c152" "c153" "c154" "c155" "c156" "c157" "c158" "c159"
[161] "c160" "c161" "c162" "c163" "c164" "c165" "c166" "c167" "c168" "c169"
[171] "c170" "c171" "c172" "c173" "c174" "c175" "c176" "c177" "c178" "c179"
[181] "c180" "c181" "c182" "c183" "c184" "c185" "c186" "c187" "c188" "c189"
[191] "c190" "c191" "c192" "c193" "c194" "c195" "c196" "c197" "c198" "c199"
[201] "c200" "c201" "c202" "c203" "c204" "c205" "c206" "c207" "c208" "c209"
[211] "c210" "c211" "c212" "c213" "c214" "c215" "c216" "c217" "c218" "c219"
[221] "c220" "c221" "c222" "c223" "c224" "c225" "c226" "c227" "c228" "c229"
[231] "c230" "c231" "c232" "c233" "c234" "c235" "c236" "c237" "c238" "c239"
[241] "c240" "c241" "c242" "c243" "c244" "c245" "c246" "c247" "c248" "c249"
[251] "c250" "c251" "c252" "c253" "c254" "c255" "c256" "c257" "c258" "c259"
[261] "c260" "c261" "c262" "c263" "c264" "c265" "c266" "c267" "c268" "c269"
[271] "c270" "c271" "c272" "c273" "c274" "c275" "c276" "c277" "c278" "c279"
[281] "c280" "c281" "c282" "c283" "c284" "c285" "c286" "c287" "c288" "c289"
[291] "c290" "c291" "c292" "c293" "c294" "c295" "c296" "c297" "c298" "c299"
[301] "c300" "c301" "c302" "c303" "c304" "c305" "c306" "c307" "c308" "c309"
[311] "c310" "c311" "c312" "c313" "c314" "c315" "c316" "c317" "c318" "c319"
[321] "c320" "c321" "c322" "c323" "c324" "c325" "c326" "c327" "c328" "c329"
[331] "c330" "c331" "c332" "c333" "c334" "c335" "c336" "c337" "c338" "c339"
[341] "c340" "c341" "c342" "c343" "c344" "c345" "c346" "c347" "c348" "c349"
[351] "c350" "c351" "c352" "c353" "c354" "c355" "c356" "c357" "c358" "c359"
[361] "c360" "c361" "c362" "c363" "c364" "c365" "c366" "c367" "c368" "c369"
[371] "c370" "c371" "c372" "c373" "c374" "c375" "c376" "c377" "c378" "c379"
[381] "c380" "c381" "c382" "c383" "c384" "c385" "c386" "c387" "c388" "c389"
[391] "c390" "c391" "c392" "c393" "c394" "c395" "c396" "c397" "c398" "c399"
[401] "c400" "c401" "c402" "c403" "c404" "c405" "c406" "c407" "c408" "c409"
[411] "c410" "c411" "c412" "c413" "c414" "c415" "c416" "c417" "c418" "c419"
[421] "c420" "c421" "c422" "c423" "c424" "c425" "c426" "c427" "c428" "c429"
[431] "c430" "c431" "c432" "c433" "c434" "c435" "c436" "c437" "c438" "c439"
[441] "c440" "c441" "c442" "c443" "c444" "c445" "c446" "c447" "c448" "c449"
[451] "c450" "c451" "c452" "c453" "c454" "c455" "c456" "c457" "c458" "c459"
[461] "c460" "c461" "c462" "c463" "c464" "c465" "c466" "c467" "c468" "c469"
[471] "c470" "c471" "c472" "c473" "c474" "c475" "c476" "c477" "c478" "c479"
[481] "c480" "c481" "c482" "c483" "c484" "c485" "c486" "c487" "c488" "c489"
[491] "c490" "c491" "c492" "c493" "c494" "c495" "c496" "c497" "c498" "c499"
[501] "c500" "c501" "c502" "c503" "c504" "c505" "c506" "c507" "c508" "c509"
[511] "c510" "c511" "c512" "c513" "c514" "c515" "c516" "c517" "c518" "c519"
[521] "c520" "c521" "c522" "c523" "c524" "c525" "c

### At this point, we can go into Python for further processing
- - -


In [2]:
setwd('/Volumes/yjiao-1/phylowgs2/output/')

In [9]:
goi <- read.table('goi_from_moshe_20160222.csv', header=TRUE, sep=',')

In [10]:
head(goi)

,Gene.Name,Popular.Name,Function,Location,Category,Pathway,Chr,Pos
1,PSMB8,LMP7,"Catalytic unit of the immunoproteosome, suited for the production of peptides for the MHC I antigen presentation",6p21.32,Genes involved in the processing of class I HLA peptides- catalytic units of the immunoproteosome- should have either deletions or mutated,HLA,NA,NA
2,PSMB9,LMP2,"Catalytic unit of the immunoproteosome, suited for the production of peptides for the MHC I antigen presentation",6p21.32,Genes involved in the processing of class I HLA peptides- catalytic units of the immunoproteosome- should have either deletions or mutated,HLA,NA,NA
3,PSMB10,LMP10 (MECL1),"Catalytic unit of the immunoproteosome, suited for the production of peptides for the MHC I antigen presentation",16q22.1,Genes involved in the processing of class I HLA peptides- catalytic units of the immunoproteosome- should have either deletions or mutated,HLA,NA,NA
4,ERAP1,ERAP1,involved in trimming HLA class I-binding precursors so that they can be presented on MHC class I molecules,5q15,Peptide triming for generation of most HLA-I binding protiens= deletion or mutated,HLA,NA,NA
5,ERAP2,ERAP2,Acts as a monomer or hetrodimer with ERAP1,5q15,Peptide triming for generation of most HLA-I binding protiens= deletion or mutated,HLA,NA,NA
6,TPP2,TPP2,The protein has a specialized function that is essential for some MHC class I antigen presentation,13q33.1,Peptide triming for generation of most HLA-I binding protiens= deletion or mutated,HLA,NA,NA
